In [27]:
%cd /home/jaeheonshim/music-vibes

/home/jaeheonshim/music-vibes


In [28]:
import torch
import numpy as np
import pandas as pd
from torch import nn
from torch.optim.lr_scheduler import CosineAnnealingLR
import torch.nn.functional as F
from torch.utils.data import random_split, DataLoader, Subset
import librosa
from tqdm import tqdm
from torchvggish import vggish_input

from vibenet.dataset import FMAWaveformDataset
from vibenet.models.teacher import PANNsMLP
from vibenet import labels

In [29]:
device = 'cpu'

model = PANNsMLP()
model = model.to(device)
data = torch.load('checkpoints/pretrained_PANN_best.pt')
model.load_state_dict(data['state_dict'])

<All keys matched successfully>

In [30]:
def eval(y):
    model.eval()

    with torch.inference_mode():
        pred = model(torch.tensor(y).unsqueeze(0).to(device))
        
    pred['acousticness'] = F.sigmoid(pred['acousticness'])
    pred['instrumentalness'] = F.sigmoid(pred['instrumentalness'])
    pred['liveness'] = F.sigmoid(pred['liveness'])
    pred['speechiness'] = torch.clamp(pred['speechiness'], 0, 1)
    pred['danceability'] = torch.clamp(pred['danceability'], 0, 1)
    pred['energy'] = torch.clamp(pred['energy'], 0, 1)
    pred['valence'] = torch.clamp(pred['valence'], 0, 1)

    return pred

In [37]:
y, sr = librosa.load("data/samples/07. Green Day - Basket Case.mp3", sr=32000)

pred = eval(y)

for i, label in enumerate(labels):
    print(f"{label:<20}{pred[label].item()}")

Note: Illegal Audio-MPEG-Header 0x00000000 at offset 7539093.
Note: Trying to resync...
Note: Hit end of (available) data during resync.


acousticness        0.16333408653736115
danceability        0.37390705943107605
energy              0.8591692447662354
instrumentalness    0.26747748255729675
liveness            0.2060476541519165
speechiness         0.06879909336566925
valence             0.5808300971984863
